# Modelos de clasificación Random Forest para el análisis de simulaciones de N-cuerpos.

El presente cuaderno de trabajo tiene el propósito de ilustrar una aplicación del algoritmo de Random Forest para analizar simulaciones numéricas de formación de estructura en cosmología. La motivación principal de este método (y en general de los trabajos de machine learning sobre simulaciones) es ahorrar el enorme costo computacional de correr y procesar estas simulaciones. Los objetivos específicos de este cuaderno son:

- Plantear el problema de formación de halos de materia oscura en términos de un problema de clasificación.
- Ejemplificar un flujo de trabajo estándar en problemas de machine learning.
- Identificar los hiperparámetros que intervienen en el desarrollo de un modelo de Random Forest y su impacto en el performance del clasificador.
- Discutir las principales métricas usadas para evaluar el performance de un clasificador.
- Presentar un método de optimización de hiperparámetros.

El notebook está preparado para correr con un mínimo de bibliotecas de Python (3+), entre las que se destacan:

- Pandas
- Seaborn
- Scikit-learn

A lo largo de este cuaderno se plantean preguntas en <font color='red'> rojo <font color='black'> que tienen como objetivo motivar la discusión sobre los resultados que se obtendrán al correr algunas de las celdas. Siéntanse con la confianza de modificar libremente el código y discutir cualquier duda a:
    
- Jazhiel Chacón: jazhielchacon@gmail.com
- Erick Almaraz : erickalmaraz@gmail.com
- Alberto Vázquez: javazquez@icf.unam.mx
    
    
Bienvenidos!

# Bibliotecas

In [ ]:
%matplotlib inline

import math
import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
from IPython.display import Image
import seaborn as sns
sns.set_palette('tab10')

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve

# Planteamiento del problema

Las simulaciones de N-cuerpos sirven de laboratorios numéricos para estudiar el impacto de modelos de materia/energía oscura en los procesos de formación de estructuras cósmicas:

<table><tr>
<td> 
  <p align="center" style="padding: 10px">
    <img alt="Forwarding" src="./z49.png" width="320">
    <br>
    <em style="color: grey">$z=49$</em>
  </p> 
</td>
<td> 
  <p align="center">
    <img alt="Routing" src="./z4.png" width="420">
    <br>
    <em style="color: grey">$z=4$</em>
  </p> 
</td>
<td> 
  <p align="center">
    <img alt="Routing" src="./z1.png" width="520">
    <br>
    <em style="color: grey">$z=1$</em>
  </p> 
</td>    
<td> 
  <p align="center">
    <img alt="Routing" src="./z0.png" width="620">
    <br>
    <em style="color: grey">$z=0$</em>
  </p> 
</td>        
</tr></table>

Sin embargo, <em> correr este tipo de cálculos requiere muchos recursos computacionales </em>. Con los avances en materia de cómputo y métodos de análisis de la información, resulta importante desarrollar aplicaciones que permitan resolver este tipo de limitaciones. 

En su forma más simple, una simulación de N-cuerpos consta de una colección grande de partículas para las cuales tenemos su posición y velocidad. A continuación, presentamos una muestra de $100,000$ partículas de materia oscura de una simulación que contiene $512^3=134,217,728$.  

In [ ]:
sim49_df = pd.read_pickle('./macss2021_particles_z49.pkl')
sim49_df.head()

Aquí, <em> particle_ID </em> es un identificador que nos ayuda a seguir cada una de ellas a diferentes tiempos. En este caso, los datos de la tabla corresponden al estado cinemático de las partículas en $z=49$, cuando las perturbaciones de la materia seguían siendo relativamente pequeñas. <em> Estas son las condiciones iniciales de la simulación.</em>

En los estudios de formación de estructura, una cantidad importante a determinar es la <em> función de masa de halos (HMF)</em>, la cual corresponde a la cantidad de halos por volumen (comóvil) por unidad de masa. Esta cantidad es sensible a la naturaleza de la materia/energía oscura, de modo que su obtención es importante para constreñir la viabilidad de modelos alternativos. Para obtener la HMF es necesario:

- Preparar las condiciones iniciales de la simulación
- Correr el código de N-cuerpos (<em> esto es lo costoso </em>)
- Postprocesar los resultados para encontrar, en este caso, los halos que se forman al final de la simulación (usualmente la época actual)

Con el auge de los métodos de inteligencia artificial, podríamos preguntarnos si es posible predecir, aunque sea de manera aproximada, diversos aspectos de una simulación a partir de las condiciones iniciales sin correr el código de N-cuerpos. Si es así, esto podría representar grandes ahorros.


<font color='blue'> En este cuaderno seremos un poco más modestos en nuestras pretenciones y nos enfocaremos a predecir si una partícula termina en un halo por encima de cierta masa umbral ($M_{th}$) a partir de la información de las condiciones iniciales. <font color='black'>
    
Comencemos.

# Feature engineering

La simulación que analizaremos corresponde a un modelo $\Lambda$CDM que corre de $z=49$ a $z=0$. Los parámetros más importantes que necesitamos conocer son:
$$N_{part}=512^3,$$ 

$$L_{box} = 200h^{-1} Mpc,$$ 

$$h=0.6768.$$ 

La masa de cada partícula de la simulación es $M_{part}=5.05\times 10^{9} h^{-1}M_{\odot}$ y la masa umbral que elegimos es $M_{th}=1.21\times 10^{12} h^{-1}M_{\odot}$. Consideraremos el intervalo de masas $M_{low}=100M_{part}$ y $M_{top}=1.5\times 10^{5}M_{part}$:

In [ ]:
Mpart = 5.05026e+09        #masa de las partículas (M_\odot h^{-1}) en la simulaciones
Mlow = 100*Mpart           #cota inferior para M 
Mtop = 150000*Mpart        #cota superior para M

#umbral (threshold) de masa para los halos (M_\odot h^{-1}) para definir las clases IN & OUT
Mth = 1.21e+12             

Si vemos la tabla de arriba, notamos de inmediato que las posiciones y las velocidades son en sí mismas atributos poco informativos. Esto es la norma en cualquier modelo de machile learning. <em> En general, antes de desarrollar un modelo es necesario preprocesar los datos de entrada para construir atributos que puedan ser ingestados en el modelo. Sin eso, los modelos simplemente no funcionan.</em>

Para este ejemplo, tomaremos el promedio de la sobredensidad centrada en cada partícula a diferentes escalas $R$:

$$\delta(x;R)=\int \delta(x') W_{TH}(x-x';R^3) d^3x',$$

donde $W_{TH}$ es una función ventana top-hat con radio característico dado por $R$ y masa de suavizado $M_{s}=\bar{\rho}\frac{4}{3} \pi R^3$, con $\bar{\rho}$ la densidad media de materia. No necesitamos entrar en detalles, solo recordar que  

$$M_{s} \propto R^3,$$ 

y por lo tanto masas más grandes corresponden a escalas más grandes.</em> Entonces, $\delta(x;R)$ contiene la información del campo de densidad en la vecindad de cada partícula. De esta manera, buscaremos predecir si una partícula termina en un halo de masa mayor a $M_{th}$ tomando la sobredensidad suavizada en diez escalas entre $M_{low}=100M_{part}$ y $M_{top}=1.5\times 10^{5}M_{part}$. <font color='red'> ¿Podemos pensar en otros atributos (como por ejemplo, usar la información de la velocidad)? <font color='black'>.
    
Para encontrar el valor de $\delta(x;R)$ podemos usar un software como <em> pynbody </em> o <em> pylians </em> (ver referencias). Nosotros aquí presentamos el resultado para las partículas de la tabla de las condiciones iniciales:

In [ ]:
data_df = pd.read_pickle('./macss2021_dataset.pkl')
data_df.head()

Entonces, cada partícula está caracterizada por un vector de diez componentes ($dr_i$) y por una etiqueta (label) que indica si:

- La partícula cae en un halo de masa mayor a $M_{th}$ en $z=0$: label=1 (IN)

- La partícula NO cae en un halo de masa mayor a $M_{th}$ o permanece libre en $z=0$: label=0 (OUT)

# Análisis exploratorio

Antes de desarrollar un modelo, es importante explorar los datos para ir desarrollando una intuición del problema. Primeramente podemos determinar cuántas partículas tienen label=1 y cuántas tienen label=0:

In [ ]:
in_df  = data_df[data_df['label']==1]
out_df = data_df[data_df['label']==0]

In [ ]:
ntotal = len(data_df)
nin    = len(in_df)
nout   = len(out_df)

print('Total de partículas            : ',ntotal)
print('Partículas en halo > M_th (IN) : ',nin)
print('Partículas en halo < M_th (OUT): ',nout)

Por la naturaleza aleatoria de $\delta(x;R)$, podemos obtener el promedio sobre todas las muestras y el máximo valor de esta cantidad para cada una de las escalas:

In [ ]:
temp_df = data_df.loc[:,data_df.columns!='label'].copy()
temp_df.describe().loc[['mean','max']]

<font color='red'> ¿Qué observamos? ¿tiene esto sentido?. <font color='black'> (<em> Hint: recordemos la relación entre $M_s$ y $R$ y el hecho de que el universo se hace más homogéneo a esclas mayores </em>).
    
Luego, podemos graficar la distribución de estas cantidades con el propósito de detectar una tendencia en los datos:

In [ ]:
fig, axs = plt.subplots(nrows=2,ncols=5,figsize=(20,8))    
fig.suptitle('Distribución de la sobredensidad para partículas IN & OUT en diferentes escalas',fontsize=16)

dr_cols = data_df.columns[:-1]

for idx in range(len(dr_cols)):
    temp_col = dr_cols[idx]
    
    if idx < 5:
        sns.distplot(in_df[temp_col],ax=axs[0,idx],kde=True,color='C0',hist=True,label='IN')
        sns.distplot(out_df[temp_col],ax=axs[0,idx],kde=True,color='C1',hist=True,label='OUT')
        axs[0,idx].axvline(x=0,color='k',linestyle=':')
        axs[0,idx].set_xlabel(temp_col,fontsize=15)
    else:
        sns.distplot(in_df[temp_col],ax=axs[1,idx-5],kde=True,color='C0',hist=True,label='IN')
        sns.distplot(out_df[temp_col],ax=axs[1,idx-5],kde=True,color='C1',hist=True,label='OUT')
        axs[1,idx-5].axvline(x=0,color='k',linestyle=':')
        axs[1,idx-5].set_xlabel(temp_col,fontsize=15)
        
    if idx == 0:
        axs[0,idx].set_ylabel('Densidad de probabilidad',fontsize=15)
        axs[0,idx].legend()
    if idx == 5: 
        axs[1,idx-5].set_ylabel('Densidad de probabilidad',fontsize=15)
        axs[1,idx-5].legend()

plt.tight_layout(rect=[0, 0.03, 1, 0.95])

<font color='red'> ¿Qué observamos? ¿tiene esto sentido?. <font color='black'>
    
Luego, podemos graficar distribuciones conjuntas que nos ayuden a identificar posibles fronteras de separación entre las dos categorías:

In [ ]:
sns.pairplot(data_df,hue="label",corner=True,palette={0:'C1',1:'C0'},plot_kws={'s':7})

<font color='red'> ¿Qué observamos? <font color='black'>
    
Finalmente, vamos a obtener el comportamiento de $\delta(x;R)$ vs $M_s$ para algunas partículas de ambas categorías:

In [ ]:
fig = plt.figure(figsize=(8,5))
fig.suptitle('Sobredensidad suavizada a diferentes escalas',fontsize=15)

ax  = plt.subplot(111)

logMlow = math.log10(Mlow)
logMtop = math.log10(Mtop)
logM    = np.linspace(logMlow,logMtop,num=10,endpoint=True) 

nsamp = 4
idx_in  = random.sample(list(in_df.index),nsamp)
idx_out = random.sample(list(out_df.index),nsamp)


for i in range(nsamp):
    if i == 0:
        plt.plot(10**logM,in_df.loc[idx_in[i],in_df.columns!='label'],'-',color='C0',linewidth=1.5,label='IN')
        plt.plot(10**logM,out_df.loc[idx_out[i],out_df.columns!='label'],'-',color='C1',linewidth=1.5,label='OUT')
        plt.legend()
    else:
        plt.plot(10**logM,in_df.loc[idx_in[i],in_df.columns!='label'],'-',color='C0',linewidth=1.5)
        plt.plot(10**logM,out_df.loc[idx_out[i],out_df.columns!='label'],'-',color='C1',linewidth=1.5)


for i in range(10):
    plt.axvline(x=10**logM[i],color='black',linestyle='dotted',linewidth=0.5)    

plt.hlines(0,Mlow,Mtop,colors='black',linestyles='dashed',linewidth=1)

plt.xlabel('$M_s (M_\odot h^{-1})$',fontsize=15)
plt.xscale('log')
plt.xlim([Mlow,Mtop])
plt.ylabel('$\delta_R$',fontsize=15)
minorLocatorY = AutoMinorLocator()
ax.yaxis.set_minor_locator(minorLocatorY)
plt.tick_params(which='major',length=6)
plt.tick_params(which='minor',length=3,color='black')

<font color='red'> ¿Qué observamos? <font color='black'>. <em>Aconsejamos correr esto n-veces para ver si detectamos una tendencia. Siéntanse con la confianza de comentar estos resultados al terminar el taller ;)

# Construcción del conjunto de entrenamiento y del conjunto de prueba

Para desarrollar un modelo de machine learning, es necesario hacer una partición en los datos en al menos dos conjuntos:

- conjunto de entrenamiento (training): son los datos que se usarán para entrenar el modelo
- conjunto de prueba (test): son los datos con los que se evaluará el desempeño del modelo

Es importante señalar que los datos del conjunto de prueba no pueden ser utilizados de ninguna manera para entrenar el modelo. En la vida real se usa un tercer conjunto llamado <em> conjunto de validación (validation) </em> con el cual se pueden ajustar los hiperparámetros de un modelo (ver abajo), de modo que el conjunto de prueba se usa sólo para testear el modelo final, sin hacer un ajuste posterior. Aquí solo tomaremos dos conjuntos.

Para este ejercicio, tomaremos solo 50000 partículas y reservaremos el 70% para entrenamiento y el 30% para prueba. Notemos que hemos tomado tantas partículas IN como partículas OUT, por lo que las clases están balanceadas. <font color='red'> Siéntanse libres de modificar el tamaño de la muestra (sample_size), la fracción usada para el entrenamiento e incluso hacer dataset con clases desbalanceadas. 

In [ ]:
sample_size = 50000
ftrain      = 0.7

In [ ]:
#partición del dataset en clases balanceadas

assert sample_size//2 <= len(in_df), 'sample_size/2 debe ser <= número total de IN'

sample_df = pd.concat([in_df.sample(sample_size//2,random_state=None),
                       out_df.sample(sample_size//2,random_state=None)])
#para hacer shuffling
sample_df = sample_df.sample(frac=1)

<font color='red'> ¿Cómo haríamos una muestra desbalanceada? <font color='black'>. Hemos visto que $IN/OUT \approx 0.53$, de modo que una manera de construir un dataset desbalanceado sería simplemente seleccionar N partículas al azar de data_df. <em> Hint: el método 'sample' es de mucha ayuda </em>.

In [ ]:
#Introducir código aquí
#sample_df = 

In [ ]:
arrays = sample_df.to_numpy()
X      = arrays[:,0:-1]
y      = arrays[:,-1]

ntrain   = math.floor(ftrain*len(X))
X_train  = X[:ntrain]
y_train  = y[:ntrain]

X_test   = X[ntrain:]
y_test   = y[ntrain:]

# Modelo de Random Forest

## Configuración del modelo

Para implementar una solución de machine learning, podemos desarrollar el código desde cero (aconsejable para modelos sencillos cuando uno está aprendiendo) o bien, usar bibliotecas que ya tienen construidos los modelos y que están optimizadas para correr más eficientemente. Nosotros usaremos la implementación del modelo de Random Forest de la biblioteca <em> scikit-learn </em>, la cual es una biblioteca de amplio uso en materia de machine learning. Otras bibliotecas que pueden ser de interés son <em> PyTorch, TensorFlow & Keras </em>.

En scikit-learn, un modelo de Random Forest se configura a través del método <em> RandomForestClassifier </em>. Existen muchas opciones de configuración cuya descripción va más allá del propósito de este cuaderno de trabajo. En nuestro caso las más importantes son:

- n_estimators: número de árboles de decisión del modelo
- criterion: criterio de partición. Puede ser 'gini' o 'entropy'
- max_depth: profundidad máxima de los árboles (None significa que los árboles se extenderán hasta obtener elementos de una misma clase y alcanzar un mínimo en el número de muestras)
- max_fratures: número de atributos a considerar para hacer una partición (auto significa sqrt{n_features} = 3 para nuestro problema)
- max_samples: número de muestras a tomar de X_train para entrenar un árbol. None significa todas.
- bootstrap: Uso de bootstrap para construir las muestras con las que se entrenarán los árboles. Es aconsejable que siempre sea True

Otros parámetros de configuración usados para control y debugging son:
- n_jobs: número de jobs en paralelo (-1 significa que se usarán todos los procesadores de nuestra máquina)
- random_state: Semilla aleatoria. Se puede usar un entero para obtener resultados reproducibles
- verbose: bandera para controlar el desplegado de mensajes (0 significa modo silencioso)

<font color='red'> Siéntanse con la libertad de modificar estos hiperparámetros a gusto. Más adelante volveremos a esta cuestión. <font color='black'>

¿Qué motiva la elección de estos valores? A priori, no hay nada que nos diga que tal o cual valor es más apropiado para hacer que un modelo tenga un mejor desempeño. De hecho, uno de los problemas de desplegar un modelo de machine learning es encontrar la combinación de estos <em> hiperparámetros </em> que optimice el desempeño del modelo. De momento, correremos nuestro modelo con esta configuración:

In [ ]:
model = RandomForestClassifier(n_estimators=100,
                               criterion='entropy',
                               max_depth=None,
                               max_features='auto',
                               max_samples=None,
                               bootstrap=True,                               
                               n_jobs=None,
                               random_state=None,
                               verbose=0)

## Entrenamiento

Una vez que ya tenemos preparados los datos y configurado nuestro modelo, podemos echar a andar el proceso de entrenamiento. Dependiendo del tipo de modelo, su complejidad y el hardware usado, esto puede tardar en ejecutarse.

In [ ]:
model.fit(X_train,y_train)

# Resultados

## Métricas de desempeño

Para evaluar el desempeño de nuestro clasificador, podemos definir cantidades que comparen las predicciones con las etiquetas reales de las muestras del conjunto de prueba. Al respecto, recordemos que en las clasificaciones binarias tenemos una clase positiva (en nuestro caso, IN o 1) y una clase negativa (OUT o 0). Al momento de hacer las predicciones del clasificador sobre una muestra, podemos tener los siguientes casos:  

- Verdaderos positivos (TP): son las muestras cuya etiqueta real es positiva (IN) y que han sido clasificadas como positivas por el clasificador.
- Falsos positivos (FP): son las muestras cuya etiqueta real es negativa (OUT), pero que han sido clasificadas como positivas por el clasificador.
- Falsos negativos (FN): son las muestras cuya etiqueta real es positiva (IN), pero que han sido clasificadas como negativas por el clasificador.
- Verdaderos negativos (TN): son las muestras cuya etiqueta real es negativa (OUT) y que han sido clasificadas como negativas por el clasificador.

Dependiendo del problema, obtener Falsos Positivos podría ser más grave que obtener Falsos Negativos y visceversa, de modo que en ese tipo de situaciones estaríamos mayormente interesados en reducir uno por sobre el otro (por ejemplo, <font color='red'>¿qué efectos tendría una prueba errónea de VIH? <font color='black'>). 
    
A continuación calcularemos las métricas más usadas en este tipo de problemas. Para aterrizar los conceptos, imaginemos que hemos desarrollado un clasificador de imágenes de gatos. En dicho clasificador nuestras muestras se dividen en dos categorías: 'gato' (clase positiva) y 'no-gato' (clase negativa).
    
    
Primero empezamos obteniendo las predicciones del clasificador y las etiquetas reales de las muestras del conjunto de prueba:

In [ ]:
y_pred = model.predict(X_test)
print('Predicciones ({})    : {}'.format(len(y_pred),y_pred))
print('Etiquetas reales ({}): {}'.format(len(y_test),y_test))

<font color='red'> ¿Podemos ver alguna discrepancia en los ejemplos desplegados?, si es así, ¿qué tipo de errores vemos (FP, FN)? 
    
<font color='black'> La siguiente función calcula el número de TP,TN,FP & FN. Los argumentos son las etiquetas reales (y_test) y las predicciones del clasificador (y_pred). 

In [ ]:
def performance(y_test,y_pred):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    
    for idx in range(len(y_test)):
        if y_test[idx]==y_pred[idx]:
            if y_test[idx]==1:
                TP += 1
            else:
                TN += 1
        else:
            if y_test[idx]==1:
                FN += 1
            else:
                FP += 1
    
    return TP, TN, FP, FN

In [ ]:
TP,TN,FP,FN = performance(y_test,y_pred)

#### a) Accuracy

La exactitud (accuracy) nos indica la fracción de predicciones acertadas respecto al total:

$$accuracy=\frac{TP+TN}{TP+FP+TN+FN}$$

En nuestro ejemplo de los gatos, la exactitud responde a la siguiente pregunta: <em> ¿cuántas de nuestras clasificaciones son correctas? </em>

La exactitud es un buen indicador del desempeño de un clasificador, si las muestras están balanceadas, es decir, si la cantidad de muestras con una etiqueta es aproximadamente la misma que la cantidad de muestras con la otra (en nuestro caso, el número de partículas IN tendría que ser similar al número de partículas OUT). En muchas situaciones esta condición no se cumple, por lo que $accuracy \approx 1$ resulta en un falso optimismo sobre el desempeño del clasificador. <font color='red'>¿Podríamos dar un ejemplo de este caso?, ¿por qué la exactitud no sería una buena métrica? <font color='black'> <em> Hint: consideremos el problema de detección de fraudes en transacciones bancarias. </em>

In [ ]:
acc_test = model.score(X_test,y_test)
acc_test

#### b) Precision

El clasificador va a clasificar cierto número de muestras con la clase positiva (TP, FP). De estas muestras, ¿cuántas son en realidad de la clase positiva? (TP). La precisión o pureza (precision) indica este dato:
$$ precision = \frac{TP}{TP+FP} $$

En nuestro ejemplo de los gatos, podemos pensar la precisión en términos de la siguiente pregunta: <em> de todos los objetos clasificados como gatos por nuestro clasificador, ¿cuántos son en realidad gatos? </em>

In [ ]:
prec_test = float(TP/(TP+FP))
prec_test

#### c) Recall

La sensibilidad (recall) es una métrica que mide la capacidad del clasificador de identificar las muestras de la clase positiva:

$$ recall = \frac{TP}{TP+FN} $$

En nuestro ejemplo de los gatos, el recall responde a la siguiente pregunta: <em> de todos los gatos en nuestra muestra, ¿cuántos pueden ser identificados por el clasificador? </em>. El recall también es conocido como la True Positive Rate (TPR).

In [ ]:
rec_test = float(TP/(TP+FN))
rec_test

#### d) Contamination

La contaminación (contamination) mide la cantidad de falsos positivos sobre el total de muestras de la clase negativa:

$$ contamination = \frac{FP}{FP+TN} $$

En nuestro ejemplo de los gatos, la contaminación responde a la siguiente pregunta: <em> de todos los objetos que NO son gatos, ¿cuántos de ellos fueron incorrectamente clasificados como gatos? </em>. La contaminación es también conocida como la False Positive Rate (FPR).

In [ ]:
cont_test = float(FP/(FP+TN))
cont_test

#### e) Specificity

La especificidad (specificity) mide la capacidad del clasificador de identificar las muestras de la clase negativa. En pocas palabras, la especificidad es el recall, pero para la clase negativa:

$$ specificity = \frac{TN}{TN+FP} $$

En nuestro ejemplo de los gatos, la especificidad responde a la siguiente pregunta: <em> de todos los no-gatos en nuestra muestra, ¿cuántos de ellos pueden ser identificados por el algoritmo? </em>

In [ ]:
spec_test = float(TN/(TN+FP))
spec_test 

## Curva ROC y AUC

La curva ROC (Receiver Operating Characteristic) y el área debajo de ésta (AUC) es una métrica de mucho uso para medir el desempeño de un clasificador binario. Esta gráfica se construye calculando la TPR y la FPR para diferentes umbrales de discriminación (el umbral de discriminación estándar es 0.5). A manera de referencia, tenemos:

.- AUC = 1.0: clasificador perfecto

.- AUC = 0.99 - 0.9: muy buen clasificador

.- AUC = 0.89 - 0.8: buen clasificador

.- AUC = 0.79 - 0.7: clasificador regular 

.- AUC = 0.69 - 0.6: clasificador malo

.- AUC = 0.50: clasificador aleatorio (el modelo no está aprendiendo)

In [ ]:
probs_IN = model.predict_proba(X_test)[:,1]
#thres[0] se fija arbitrariamente
fpr,tpr,thres = roc_curve(y_test,probs_IN)   
auc_roc  = roc_auc_score(y_test,probs_IN)

<font color='red'> ¿Qué valores tiene thres?, ¿qué impacto tiene variar el umbral de discriminación en la TPR (recall) y en la FPR (contaminación)?

In [ ]:
fig = plt.figure(figsize=(8,5))
fig.suptitle('Curva ROC para el clasificador Random Forest (AUC={:.3f})'.format(auc_roc),fontsize=14)

ax  = plt.subplot(111)
ax.plot(fpr,tpr,color='C0',linewidth=1.5,linestyle='-',label='Random Forest')
ax.plot(fpr,fpr,color='C3',linewidth=1.5,linestyle='--',label='Random Prediction')
ax.set_xlabel('False Positive Rate (Contamination)',fontsize=13)
ax.set_xlim([-0.01,1])   
ax.set_ylabel('True Positive Rate (Recall)',fontsize=13)
ax.set_ylim([0,1.05])
ax.legend(loc='lower right')
minorLocatorX = AutoMinorLocator()
ax.xaxis.set_minor_locator(minorLocatorX)
minorLocatorY = AutoMinorLocator()
ax.yaxis.set_minor_locator(minorLocatorY)
plt.tick_params(which='major',length=6)
plt.tick_params(which='minor',length=3,color='black')

### Matriz de confusión

En los problemas de clasificación, resulta útil desplegar los resultados de TP, TN, FP & FN en un arreglo que se conoce como <em> matriz de confusión </em>. La matriz de confusión nos ayuda a identificar en dónde el modelo está acertando y en dónde está fallando. Las columnas representan las predicciones y los renglones las etiquetas reales. En scikit-learn, la función <em> confusion_matrix </em> nos ayuda a calcular la matriz de confusión a partir de la información de las etiquetas reales y de las predicciones. La opción <em> normalize </em> despliega los resultados normalizando respecto a las columnas, los renglones, el tamaño de la muestra o sin normalizar (None). <font color='red'> Sería bueno correr la siguiente celda y averiguar el significado de cada una de estas opciones.  

In [ ]:
pred_class = model.predict(X_test)
confusion_matrix(y_true=y_test,y_pred=pred_class,normalize=None)

Podemos desplegar estos resultados de una forma visualmente más atractiva:

In [ ]:
def plot_binary_confusion_matrix(model,X_test,y_test,normalize):
    '''
    Plots the confusion matrix for binary IN/OUT
    classification.
    
    Inputs:
        model: a trained model
        testX: test set in numpy array format
        testy: ground true labels of the objects

    Output:
        binary_confusion_matrix.pdf: pdf file containing
                                     the confusion matrix
    '''
    
    #predicted class (array consisting of 0/1)
    #is not the same as predict_proba
    pred_class = model.predict(X_test)
    
    #confusion matrix via sklearn
    conf_mat = confusion_matrix(y_true=y_test,y_pred=pred_class,normalize=normalize)
    #print(conf_mat)
    
    classes = ['OUT','IN']

    fig, ax = plt.subplots(figsize=(6,6))

    plt.imshow(conf_mat,cmap=plt.cm.Blues,interpolation='nearest',aspect='auto')

    #x-axis
    tick_marks = np.arange(len(classes))   
    plt.xlabel('Predicted labels')
    plt.xticks(tick_marks,classes,rotation=45)
    #y-axis
    plt.ylabel('True labels')
    ax.set_yticks(tick_marks)
    ax.set_yticklabels(['OUT','IN']);
    plt.ylim([1.5,-0.5])
    #to remove the grid in the plot
    ax.grid(False)
    #loop over data dimensions to create text annotations
    for i in range(len(classes)):
        for j in range(len(classes)):
            #just to give more visibility to text annotations
            if i == j:
                color='white'
            else:
                color='black'            
            ax.text(j,i,'{:.3f}'.format(conf_mat[i, j]),ha="center",va="center",color=color)
    #export image to file
    plt.tight_layout()
    #plt.savefig('binary_confusion_matrix.pdf')

    return

In [ ]:
plot_binary_confusion_matrix(model=model,X_test=X_test,y_test=y_test,normalize='true')

<font color='red'> ¿Cómo se compara la tasa de FP con la de FN?

##  Feature Importance

Algo atractivo de los árboles de decisión es que podemos evaluar el impacto que tienen cada uno de los atributos para hacer una clasificación. Esto nos podría ayudar a identificar aquellos atributos que son más determinantes para nuestro problema. Consideremos, por ejemplo, el peso que tiene un rasgo distintivo como un tatuaje, la forma de los ojos, una cicatriz, la calvicie, etc para identificar a una persona.

Algo similar podemos hacer en nuestro problema. El atributo <em> feature_importances_ </em> asigna un score de importancia a cada uno de los atributos $dr_i$ de las muestras:

In [ ]:
feature_imp = pd.Series(model.feature_importances_,index=['dr'+str(i) for i in range(1,11)])

fig, ax = plt.subplots(figsize=(8,5))    
fig.suptitle('Features importances para el clasificador Random Forest',fontsize=15)
sns.barplot(x=feature_imp.index,y=feature_imp,palette="Blues_d")
plt.xlabel('Features',fontsize=15)
plt.ylabel('Features Importance Score',fontsize=15);

## Ajuste de hiperparámetros

Si entrenamos el modelo con los parámetros por default, veremos que la exactitud entre el training y el test set es muy diferente:

In [ ]:
print('Accuracy en train: {:.3f}'.format(model.score(X_train,y_train)))
print('Accuracy en test : {:.3f}'.format(model.score(X_test,y_test)))

Esto significa que el modelo aprendió a clasificar muy bien las muestras usadas en el entrenamiento, pero no puede ser usado clasificar con precisión muestras que <em> no ha visto </em> (o sea, el conjunto de prueba). Este problema es conocido como <em> sobreajuste </em> (overfitting) y es con toda seguridad el problema más delicado al momento de desplegar un modelo de machine learning. A final de cuentas, ¿de qué serviría entrenar un modelo si éste no pudiera ser generalizado? 

Por otro lado, ¿cómo podríamos asegurarnos que hemos elegido los hiperparámetros óptimos para nuestro modelo? 

¿Cómo podríamos resolver estos dos problemas? A continuación, presentaremos un método muy usado llamado <em> Grid Search </em>, el cual consiste en correr diferentes implementaciones del Random Forest en una cuadrícula (grid) de hiperparámetros. La idea es seleccionar justamente los hiperparámetros que den mejores resultados.

Para este problema, nuestra grid consistirá de $3 \times 3 \times 3 \times 3 = 81$ combinaciones:

In [ ]:
# define models and parameters
rf_gs        = RandomForestClassifier()
n_estimators = [100,500,1000]
max_depth    = [3,8,None]
max_features = ['auto',5,None]
max_samples  = [0.3,0.6,None]

Para atender el problema de overfitting, podemos hacer una <em> K-Fold Cross-Validation </em>. La idea de esto es que tengamos un modelo optimizado y que a la vez tenga poco sobreajuste.

Entonces, de acuerdo a los parámetros que hemos insertado (n_splits=5 & n_repeats=5), correremos un total de 81 modelos $5 \times 5 = 25$ veces, dando un total de 2025 corridas. 

<font color='red'> <b> Advertencia: dada la cantidad de modelos a evaluar y el total de k-foldings y repeticiones, la siguiente celda se tarda en ejecutar alrededor de 12hr en una máquina de doble núcleo </b> 

In [ ]:
# define grid search
grid = dict(n_estimators=n_estimators,
            max_depth=max_depth,
            max_features=max_features,
            max_samples=max_samples)
#cross-validation strategy
cv = RepeatedStratifiedKFold(n_splits=5,n_repeats=5,random_state=None)

grid_search = GridSearchCV(estimator=rf_gs,
                           param_grid=grid,
                           n_jobs=-1,
                           cv=cv,
                           scoring='accuracy',
                           error_score=0,
                           verbose=4)
grid_result = grid_search.fit(X_train,y_train)

In [ ]:
# Resultados
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

<font color='red'> Una vez que hemos encontrado los mejores hiperparámetros, podemos entrenar un modelo con ellos y comparar los resultados respecto a nuestro primer clasificador:

In [ ]:
rf_opt = RandomForestClassifier(n_estimators=#meter_valor,
                                criterion='entropy',
                                max_depth=#meter_valor,
                                max_features=#meter_valor,
                                max_samples=#meter_valor,
                                bootstrap=True,                               
                                n_jobs=None,
                                random_state=None,
                                verbose=0)

rf_opt.fit(X_train,y_train)

print('Accuracy en train: {:.3f}'.format(rf_opt.score(X_train,y_train)))
print('Accuracy en test : {:.3f}'.format(rf_opt.score(X_test,y_test)))

<font color='red'> ¿Resolvimos el problema de sobreajuste?, ¿cuál es la exactitud final del modelo optimizado? <font color='black'> Si bien nuestro modelo ya es generalizable, queda pendiente mejorar su desempeño. A este problema se le conoce como <em> bias. </em>

# Referencias

En este notebook hemos visto cómo podemos aplicar un modelo de machine learning para analizar simulaciones cosmológicas. Esperamos que este material haya sido estimulante para explorar esta línea de investigación con mucho potencial. Dejamos este conjunto de referencias e invitamos a mantenernos en contacto:

- Librerias pygadgetreader & pynbody & pylians para analizar simulaciones de N cuerpos:

https://github.com/jveitchmichaelis/pygadgetreader

https://pynbody.github.io/pynbody/

https://github.com/franciscovillaescusa/Pylians

- Random Forest en scikit-learn:

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

- Artículos en el que se basó este trabajo:

Machine learning cosmological structure formation: https://arxiv.org/abs/1802.04271

Classification algorithms applied to structure formation simulations: https://arxiv.org/abs/2106.06587

- Compilación de artículos de machine learning aplicados a cosmología:

https://github.com/georgestein/ml-in-cosmology